### Reader: DistilBERT

In [1]:
!pip install -qq datasets==2.16.1 evaluate==0.4.1
!pip install -qq accelerate==0.26.1
!apt install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 11.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2023.10.0 which is incompatible.
pathos 0.3.2 requires dill>=0.3.8, but you have dill 0.3.7 which is incompatible.
pathos 0.3.2 requires multiprocess>=0.70.16, but you have multiprocess 0.70.15 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.4 MB/s eta 0:00:00a 0:00:01
Reading package lists... Done
Building dependency

In [2]:
# login huggingface download dataset và model có sẵn
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import numpy as np
from tqdm.auto import tqdm
import collections

import torch

from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering
from transformers import TrainingArguments
from transformers import Trainer
import evaluate

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
# sử dụng mô hình 'distilbert-base-uncased' làm mô hình checkpoint
MODEL_NAME = 'distilbert-base-uncased'

# Độ dài tối đa cho mỗi đoạn văn bản sau khi được xử lý
MAX_LENGTH = 384

# Khoảng cách giữa các điểm bắt đầu của các đoạn văn bản liên tiếp
STRIDE = 128

#### Setup Dataset

In [5]:
# Download squad dataset từ HuggingFace
DATASET_NAME = 'squad_v2'
raw_datasets = load_dataset(DATASET_NAME)

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [6]:
# Load tookenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

#### Tokenize dataset

##### Tokenize train set

In [7]:
# Định nghĩa hàm preprocess_training_examples và nhận tham số examples
# là dữ liệu training
def preprocess_training_examples(examples):
    # Trích xuất danh sách câu hỏi từ examples và
    # loại bỏ các khoảng trắng dư thừa
    questions = [q.strip() for q in examples["question"]]

    # Tiến hành mã hóa thông tin đầu vào sử dụng tokenizer
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Trích xuất offset_mapping từ inputs và loại bỏ nó ra khỏi inputs
    offset_mapping = inputs.pop("offset_mapping")

    # Trích xuất sample_map từ inputs và loại bỏ nó ra khỏi inputs
    sample_map = inputs.pop("overflow_to_sample_mapping")

    # Trích xuất thông tin về câu trả lời (answers) từ examples
    answers = examples["answers"]

    # Khởi tạo danh sách các vị trí bắt đầu và kết thúc câu trả lời
    start_positions = []
    end_positions = []

    # Duyệt qua danh sách offset_mapping
    for i, offset in enumerate(offset_mapping):
        # Xác định index của mẫu (sample) liên quan đến offset hiện tại
        sample_idx = sample_map[i]

        # Trích xuất sequence_ids từ inputs
        sequence_ids = inputs.sequence_ids(i)

        # Xác định vị trí bắt đầu và kết thúc của ngữ cảnh
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx- 1

        # Trích xuất thông tin về câu trả lời cho mẫu này
        answer = answers[sample_idx]

        if len(answer['text']) == 0:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Xác định vị trí ký tự bắt đầu và kết thúc của câu trả lời
            # trong ngữ cảnh
            start_char = answer["answer_start"][0]
            end_char = answer["answer_start"][0] + len(answer["text"][0])

            # Nếu câu trả lời không nằm hoàn toàn trong ngữ cảnh,
            # gán nhãn là (0, 0)
            if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
                start_positions.append(0)
                end_positions.append(0)
            else:
                # Nếu không, gán vị trí bắt đầu và kết thúc dựa trên
                # vị trí của các mã thông tin
                idx = context_start
                while idx <= context_end and offset[idx][0] <= start_char:
                    idx += 1
                start_positions.append(idx- 1)

                idx = context_end
                while idx >= context_start and offset[idx][1] >= end_char:
                    idx-= 1
                end_positions.append(idx + 1)

    # Thêm thông tin vị trí bắt đầu và kết thúc vào inputs
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs

In [8]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [9]:
raw_datasets['train']['context'][0]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [10]:
raw_datasets['train']['question'][0]

'When did Beyonce start becoming popular?'

In [11]:
raw_datasets['train']['answers'][0]

{'text': ['in the late 1990s'], 'answer_start': [269]}

In [12]:
# Tạo một biến train_dataset và gán cho nó giá trị sau khi áp dụng hàm preprocess_training_examples lên tập dữ liệu "train"
# batched=True: Bật chế độ xử lý theo từng batch
# remove_columns: Loại bỏ các cột không cần thiết trong tập dữ liệu "train"

train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

 # In ra độ dài của tập dữ liệu "train" ban đầu và
 # độ dài của tập dữ liệu đã được xử lý (train_dataset)
len(raw_datasets['train']), len(train_dataset)

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

(130319, 131754)

##### Tokenize val_set

In [13]:
def preprocess_validation_examples(examples):
    # Chuẩn bị danh sách câu hỏi bằng cách
    # loại bỏ khoảng trắng ở đầu và cuối mỗi câu hỏi
    questions = [q.strip() for q in examples["question"]]
    # Sử dụng tokenizer để mã hóa các câu hỏi và văn bản liên quan
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Lấy ánh xạ để ánh xạ lại ví dụ tham chiếu cho từng dòng trong inputs
    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    # Xác định ví dụ tham chiếu cho mỗi dòng đầu vào và
    # điều chỉnh ánh xạ offset
    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]

        # Loại bỏ các offset không phù hợp với sequence_ids
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    # Thêm thông tin ví dụ tham chiếu vào đầu vào
    inputs["example_id"] = example_ids

    return inputs

In [14]:
# Tạo một biến validation_dataset và gán giá trị bằng việc sử dụng dữ liệu
# từ raw_datasets["validation"] sau khi áp dụng một hàm xử lý trước.
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)

# In ra độ dài của raw_datasets["validation"] và validation_dataset để so sánh.
len(raw_datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

(11873, 12134)

#### Train model

In [15]:
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
import wandb

api_key = input("API key của wandb: ")
wandb.login(key=api_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: noinaycoanh094 (noinaycoanh094-hanoi-university-of-civil-engineering). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [17]:
args = TrainingArguments(
    output_dir='distilbert-finetuned-squadv2',
    evaluation_strategy='no',
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=True
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-17-819e9d2ba7d4>:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:449: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
500,2.839200
1000,1.889900
1500,1.693300
2000,1.579100
2500,1.541100
3000,1.465600
3500,1.427300
4000,1.383200
4500,1.360200
5000,1.305000


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=24705, training_loss=1.071945691953133, metrics={'train_runtime': 9534.841, 'train_samples_per_second': 41.454, 'train_steps_per_second': 2.591, 'total_flos': 3.873165421863629e+16, 'train_loss': 1.071945691953133, 'epoch': 3.0})

In [18]:
trainer.push_to_hub(commit_message='Training complete')

events.out.tfevents.1740366557.fc2fcd9ca88f.31.0:   0%|          | 0.00/15.7k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hieuhatinh/distilbert-finetuned-squadv2/commit/c8cd494a7d04360b69ef3df73d348119e0ee6d66', commit_message='Training complete', commit_description='', oid='c8cd494a7d04360b69ef3df73d348119e0ee6d66', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hieuhatinh/distilbert-finetuned-squadv2', endpoint='https://huggingface.co', repo_type='model', repo_id='hieuhatinh/distilbert-finetuned-squadv2'), pr_revision=None, pr_num=None)

#### Evaluate model

In [2]:
!pip install evaluate

In [3]:
import evaluate
metric = evaluate.load('squad_v2')

In [4]:
N_BEST = 20 # Số lượng kết quả tốt nhất được lựa chọn sau khi dự đoán
MAX_ANS_LENGTH = 30 # Độ dài tối đa cho câu trả lời dự đoán

def compute_metrics(start_logits, end_logits, features, examples):
    # Tạo một từ điển mặc định để ánh xạ mỗi ví dụ
    # với danh sách các đặc trưng tương ứng
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature['example_id']].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example['id']
        context = example['context']
        answers = []

        # Lặp qua tất cả các đặc trưng liên quan đến ví dụ đó
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]['offset_mapping']

            # Lấy các chỉ số có giá trị lớn nhất cho start và end logits
            start_indexes = np.argsort(start_logit)[-1:-N_BEST-1:-1].tolist()
            end_indexes = np.argsort(end_logit)[-1:-N_BEST-1:-1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Bỏ qua các câu trả lời
                    # không hoàn toàn nằm trong ngữ cảnh
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Bỏ qua các câu trả lời có độ dài > max_answer_length
                    if end_index-start_index + 1 > MAX_ANS_LENGTH:
                        continue

                    # Tạo một câu trả lời mới
                    text = context[
                        offsets[start_index][0]:offsets[end_index][1]
                    ]
                    logit_score = start_logit[start_index] + end_logit[end_index]
                    answer = {
                        'text': text,
                        'logit_score': logit_score,
                    }
                    answers.append(answer)

        # Chọn câu trả lời có điểm số tốt nhất
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x['logit_score'])
            answer_dict = {
                'id': example_id,
                'prediction_text': best_answer['text'],
                'no_answer_probability': 1-best_answer['logit_score']
            }
        else:
            answer_dict = {
                'id': example_id,
                'prediction_text': '',
                'no_answer_probability': 1.0
            }
            predicted_answers.append(answer_dict)

    # Tạo danh sách câu trả lời lý thuyết từ các ví dụ
    theoretical_answers = [
        {'id': ex['id'], 'answers': ex['answers']} for ex in examples
    ]
    # Sử dụng metric.compute để tính toán các độ đo và trả về kết quả
    return metric.compute(
        predictions=predicted_answers,
        references=theoretical_answers
    )

In [5]:
# Thực hiện dự đoán trên tập dữ liệu validation
predictions, _, _ = trainer.predict(validation_dataset)

# Lấy ra thông tin về các điểm bắt đầu và
# điểm kết thúc của câu trả lời dự đoán
start_logits, end_logits = predictions

# Tính toán các chỉ số đánh giá sử dụng hàm compute_metrics
results = compute_metrics(
    start_logits,
    end_logits,
    validation_dataset,
    raw_datasets["validation"]
)
results

NameError: name 'trainer' is not defined

#### Load model from huggingface-hub

In [7]:
from transformers import pipeline
PIPELINE_NAME = 'question-answering'
MODEL_NAME = 'hieuhatinh/distilbert-finetuned-squadv2'
pipe = pipeline(PIPELINE_NAME, model=MODEL_NAME)

config.json:   0%|          | 0.00/561 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
INPUT_QUESTION = 'What is my name?'
INPUT_CONTEXT = 'My name is AI Vietnam and I live in Vietnam.'
pipe(question=INPUT_QUESTION, context=INPUT_CONTEXT)


{'score': 0.9590733647346497, 'start': 11, 'end': 21, 'answer': 'AI Vietnam'}

### Retriever: Faiss

In [1]:
!pip install -qq  datasets==2.16.1 evaluate==0.4.1
!sudo apt-get install libomp-dev
!pip install -qq faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2023.10.0 which is incompatible.
pathos 0.3.2 requires dill>=0.3.8, but you have dill 0.3.7 which is incompatible.
pathos 0.3.2 requires multiprocess>=0.70.16, but you have multiprocess 0.70.15 which is incompatible.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages wil

In [2]:
import numpy as np
import collections
import torch
import faiss
import evaluate

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForQuestionAnswering
from transformers import TrainingArguments
from transformers import Trainer
from tqdm.auto import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

#### Tải bộ dữ liệu

In [3]:
DATASET_NAME = 'squad_v2'
raw_datasets = load_dataset(DATASET_NAME, split='train+validation')
raw_datasets

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 142192
})

#### Loại bỏ các mẫu không có đáp án

In [4]:
raw_datasets = raw_datasets.filter(
    lambda x: len(x['answers']['text']) > 0
)

Filter:   0%|          | 0/142192 [00:00<?, ? examples/s]

#### Khởi tạo mô hình

In [5]:
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

#### Xây dựng hàm lấy vector embedding:

In [6]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [7]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)

    return cls_pooling(model_output)

#### Xây dựng vector database

In [8]:
EMBEDDING_COLUMN = 'question_embedding'
embeddings_dataset = raw_datasets.map(
    lambda x: {
        EMBEDDING_COLUMN: get_embeddings(
            x['question']
        ).detach().cpu().numpy()[0]
    }
)

embeddings_dataset.add_faiss_index(column=EMBEDDING_COLUMN)

Map:   0%|          | 0/92749 [00:00<?, ? examples/s]

  0%|          | 0/93 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'question_embedding'],
    num_rows: 92749
})

In [9]:
input_question = 'When did Beyonce start becoming popular?'

input_quest_embedding = get_embeddings([input_question])
input_quest_embedding = input_quest_embedding.cpu().detach().numpy()

TOP_K = 5
scores, samples = embeddings_dataset.get_nearest_examples(
    EMBEDDING_COLUMN, input_quest_embedding, k=TOP_K
)

for idx, score in enumerate(scores):
    print(f'Top {idx + 1}\tScore: {score}')
    print(f'Question: {samples["question"][idx]}')
    print(f'Context: {samples["context"][idx]}')
    print()

Top 1	Score: 0.0
Question: When did Beyonce start becoming popular?
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".

Top 2	Score: 2.6135294437408447
Question: When did Beyoncé rise to fame?
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress

#### Áp dụng mô hình hỏi-đáp để trả lời câu hỏi

In [10]:
from transformers import pipeline

PIPELINE_NAME = 'question-answering'
MODEL_NAME = 'hieuhatinh/distilbert-finetuned-squadv2'
pipe = pipeline(PIPELINE_NAME, model=MODEL_NAME)

config.json:   0%|          | 0.00/561 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


In [11]:
print(f'Input question: {input_question}')
for idx, score in enumerate(scores):
    question = samples["question"][idx]
    context = samples["context"][idx]
    answer = pipe(
        question=question,
        context=context
    )
    print(f'Top {idx + 1}\tScore: {score}')
    print(f'Context: {context}')
    print(f'Answer: {answer}')
    print()

Input question: When did Beyonce start becoming popular?
Top 1	Score: 0.0
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
Answer: {'score': 0.486748069524765, 'start': 276, 'end': 286, 'answer': 'late 1990s'}

Top 2	Score: 2.6135294437408447
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an Amer